In [ ]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os


GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-pro")

In [ ]:
llm.invoke("who is the prime minister of india")

In [ ]:
from langchain_community.document_loaders import YoutubeLoader

youtube_url = "https://youtu.be/zIwLWfaAg-8?si=qsN6eA17rsIqJl8o"
loader = YoutubeLoader.from_youtube_url(youtube_url=youtube_url, add_video_info=False)
docs = loader.load()

In [ ]:
docs


In [ ]:
template = f"""
You are a helpful assistant programmed to generate questions based on any text provided. For every chunk of text you receive, you're tasked with designing 10 distinct questions. Each of these questions will be accompanied by 3 possible answers: one correct answer and two incorrect ones. 
provided text is as follows:
{docs}

For clarity and ease of processing, structure your response in a way that emulates a Python list of lists. 

Your output should be shaped as follows:

1. An outer list that contains 5 inner lists.
2. Each inner list represents a set of question and answers, and contains exactly 4 strings in this order:
- The generated question.
- The correct answer.
- The first incorrect answer.
- The second incorrect answer.

Your output should mirror this structure:
[
    ["Generated Question 1", "Correct Answer 1", "Incorrect Answer 1.1", "Incorrect Answer 1.2"],
    ["Generated Question 2", "Correct Answer 2", "Incorrect Answer 2.1", "Incorrect Answer 2.2"],
    ...
]

It is crucial that you adhere to this format as it's optimized for further Python processing.

"""

a = llm.invoke(template).content

print(a)

In [ ]:
from typing import List

import getpass
import os
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Provide your Google API Key")

model = ChatGoogleGenerativeAI(model="gemini-pro")
que_no = 10

q = f"""
You are a helpful assistant programmed to generate questions based on any text provided. For every chunk of text you receive, you're tasked with designing {que_no} distinct questions. Each of these questions will be accompanied by 3 possible answers: one correct answer and two incorrect ones. 
provided text is deliminated in tripple ticks:
```{docs}```

For clarity and ease of processing, structure your response in a way that emulates a Python list of lists. 

Your output should be shaped as follows:

1. An outer list that contains {que_no} inner lists.
2. Each inner list represents a set of question and answers, and contains exactly 4 strings in this order:
- The generated question.
- The correct answer.
- The first incorrect answer.
- The second incorrect answer.

Your output should mirror this structure:
[
    ["Generated Question 1", "Correct Answer 1", "Incorrect Answer 1.1", "Incorrect Answer 1.2"],
    ["Generated Question 2", "Correct Answer 2", "Incorrect Answer 2.1", "Incorrect Answer 2.2"],
    ...
]

It is crucial that you adhere to this format as it's optimized for further Python processing.

"""

parser = JsonOutputParser()

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

otpt = chain.invoke({"query":q})


In [ ]:
otpt

In [ ]:
import random

output = [['Which model is considered the grandfather of all life cycle models?',
  'Waterfall model',
  'Agile model',
  'Scrum model'],
 ['In the waterfall model, what is the final phase?',
  'System testing',
  'Requirements gathering',
  'Design'],
 ['What is an advantage of the waterfall model?',
  'Finding errors early',
  'Flexibility',
  'Rapid development'],
 ['What is a disadvantage of the waterfall model?',
  'Lack of flexibility',
  'High cost',
  'Long development time'],
 ['When is the waterfall model most suitable?',
  'Stable product definition and well-known domain',
  'Changing requirements and evolving technology',
  'Small and simple projects']]


def get_randomized_options(options):
    correct_answer = options[0]
    random.shuffle(options)
    return options, correct_answer


## Initial code 

In [ ]:
import random
import streamlit as st
import os
import json
from langchain_community.document_loaders import YoutubeLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model="gemini-pro")

def get_randomized_options(options):
    correct_answer = options[0]
    random.shuffle(options)
    return options, correct_answer

st.title("📚🧠Youtube Video Quiz App")


name = st.text_input("👩‍💼 Enter your name: ")
url = st.text_input("🔗 Enter your video url: ")
process =  st.checkbox("Submit and Process")

if process:
    # Initialize variables
    st.write(f"Hello, {name}!!")
    # youtube_url = "https://youtu.be/zIwLWfaAg-8?si=qsN6eA17rsIqJl8o"

    with st.spinner("Loading youtube content..."):
        loader = YoutubeLoader.from_youtube_url(youtube_url=url, add_video_info=False)
        doc = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1500,
        chunk_overlap = 150
    )
    docs = text_splitter.split_documents(doc)


    que_no = 8

    q = f"""
    You are a helpful assistant programmed to generate questions based on any text provided. For every chunk of text you receive, you're tasked with designing {que_no} distinct questions. Each of these questions will be accompanied by 3 possible answers: one correct answer and two incorrect ones. 
    provided text is deliminated in tripple ticks:
    ```{docs}```

    For clarity and ease of processing, structure your response in a way that emulates a Python list of lists. 

    Your output should be shaped as follows:

    1. An outer list that contains {que_no} inner lists.
    2. Each inner list represents a set of question and answers, and contains exactly 4 strings in this order: (it should STRICTLY contain 4 strings)
    - The generated question.
    - The correct answer.
    - The first incorrect answer.
    - The second incorrect answer.

    Your output should mirror this structure:
    [
        ["Generated Question 1", "Correct Answer 1", "Incorrect Answer 1.1", "Incorrect Answer 1.2"],
        ["Generated Question 2", "Correct Answer 2", "Incorrect Answer 2.1", "Incorrect Answer 2.2"],
        ...
    ]

    It is crucial that you adhere to this format as it's optimized for further Python processing.

    """

    parser = JsonOutputParser()

    prompt = PromptTemplate(
        template="Answer the user query.\n{format_instructions}\n{query}\n",
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )

    chain = prompt | llm | parser

    output = chain.invoke({"query":q})


    score = 0
    total_questions = len(output)
    incorrect_answers = []


    with st.form("quiz_form"):
        for i, question_data in enumerate(output, start=1):
            question, correct_answer, incorrect_answer1, incorrect_answer2 = question_data
            
            # Randomize options
            options, correct_answer = get_randomized_options([correct_answer, incorrect_answer1, incorrect_answer2])
            
            st.write(f"{i}. {question}")

            selected_option = st.radio(f"Choose the correct answer for Question {i}:", options, key=i)
            
            st.session_state[f"selected_option_{i}"] = selected_option

        submit_button = st.form_submit_button("Submit")
    

    if submit_button:
        # Iterate through each question to display the correct answer and update score
        for i, question_data in enumerate(output, start=1):
            correct_answer = question_data[1]
            selected_option = st.session_state[f"selected_option_{i}"]

            if selected_option == correct_answer:
                score += 1
            else:
                incorrect_answers.append(i)
        

        st.header("🎉Quiz Summary")

        for i in range(total_questions):
            if i+1 in incorrect_answers:
                # st.write(f"❌Que {i+1}: incorrect")
                st.write(f"❌ Question {i+1}: {output[i][0]}")
                st.write(f"Correct ans: {output[i][1]}")

            else:
                # st.write(f"✔️Que {i+1}: correct")
                st.write(f"✔️ Question {i+1}: {output[i][0]}")
                # st.write(f"Correct ans: {output[i][1]}")

        


        # Save the user information and quiz summary to a JSON file
        quiz_data = {
                    "name": name,
                    "questions": output,
                    "score": score
                }

        # Define the output file name
        output_file_name = f"result.json"

        # Check if the output file already exists
        if os.path.exists(output_file_name):
            # Load existing data from the file
            with open(output_file_name, "r") as file:
                existing_data = json.load(file)
            
            # Append new data to the existing data
            existing_data.append(quiz_data)
            
            # Write the updated data back to the file
            with open(output_file_name, "w") as file:
                json.dump(existing_data, file)
        else:
            # Write the JSON data to the file if it doesn't exist
            with open(output_file_name, "w") as file:
                json.dump([quiz_data], file)




